# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

In [1]:
import json
import re
from pathlib import Path

import dask.bag as db
import nltk
from dask.delayed import delayed
from dask.distributed import Client

In [2]:
DATA_DIR = Path('data/')
OUTPUT_DIR = Path('output/')
if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir()

In [3]:
client = Client(n_workers=4, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53425,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.86 GiB
Comm: tcp://192.168.1.133:53452,Total threads: 1
Dashboard: http://192.168.1.133:53453/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:53430,


## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [4]:
bag = db.read_text(
    DATA_DIR.joinpath('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt'),
    encoding='cp1251'
)

In [5]:
sentence_bag = bag.map(nltk.sent_tokenize).flatten()
sentence_bag.take(5)

('Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net',
 'Все книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html',
 'Эта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html',
 'Приятного чтения!',
 'Федор Михайлович Достоевский')

In [6]:
sentence_bag.map(len).take(5)

(98, 73, 93, 17, 28)

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [7]:
sentence_bag.filter(lambda x: len(x) <= 10).take(5)

('Игрок', 'Глава I', '— ton).]', '—\xa0Плюнуть?', '«Как!')

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [8]:
sentence_bag.map(len).mean().compute()

67.212271973466

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [9]:
def read_json(path):
    with open(path) as f:
        return json.load(f) * 1_000_000

In [10]:
address = db.from_delayed(delayed(read_json)(DATA_DIR.joinpath('addres-book.json')))

In [11]:
pattern = re.compile('^\d{3}-\d{2}-\d{2}$')
is_work_phone = pattern.fullmatch


def case_1():
    def reducer(res, value):
        i = j = 0
        for phone in value['phones']:
            if is_work_phone(phone['phone']):
                i += 1
            else:
                j += 1
        return res[0] + i, res[1] + j

    return address.fold(reducer, initial=(0, 0)).compute()


def case_2():
    def reducer(res, value):
        if is_work_phone(value['phone']):
            return res[0] + 1, res[1]
        return res[0], res[1] + 1

    return address.map(lambda x: x['phones']).flatten().fold(reducer, initial=(0, 0)).compute()

In [12]:
%time case_1()

Wall time: 6.31 s


(6000000, 7000000)

In [13]:
%time case_2()

Wall time: 10.1 s


(6000000, 7000000)

## Лабораторная работа 12

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [14]:
reviews_bag = db.read_text(DATA_DIR.joinpath('reviews_full/*.json')).map(json.loads)
reviews_bag.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [15]:
pattern = re.compile(r'^.*reviews_(\d+)\.json$')


def in_dict_with_rating(element: tuple[str, str]) -> dict:
    data, path = element
    rating = int(pattern.match(path).groups()[0])
    return json.loads(data) | {'rating': rating}

In [16]:
reviews_bag = db.read_text(
    DATA_DIR.joinpath('reviews_full/*.json'),
    include_path=True
).map(in_dict_with_rating)
reviews_bag.take(3)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [17]:
%%time
reviews_bag.count().compute()

Wall time: 54.3 s


9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [18]:
%%time
reviews_bag_sample: db.Bag = reviews_bag.filter(lambda x: x['date'][:4] in {'2014', '2015'}).persist()
reviews_bag_sample.take(3)

Wall time: 4.47 s


({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, s

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [19]:
extra_sub = re.compile(r'[^a-z ]')
s_sub = re.compile(r' +')


def preproc(x: dict) -> dict:
    review = x['review'].strip().lower()
    x['review'] = s_sub.sub(' ', extra_sub.sub('', review))
    return x



In [20]:
preproc_reviews_bag_sample: db.Bag = reviews_bag_sample.map(preproc).persist()
preproc_reviews_bag_sample.take(3)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time welltoasted bread is the key really toast it i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor i used sourdough bread medium cheddar cheese fast fun great idea made for hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic added the olives under the cheese and baked uncovered for the minutes served with pico de gallo sour cream and avocado chunks fantastic thanks for sharing l

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [ ]:
preproc_reviews_bag_sample.count().compute()

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [ ]:
preproc_reviews_bag_sample.foldby(
    lambda x: x['rating'],
    lambda accum, _: accum + 1,
    initial=0
).compute()

In [ ]:
preproc_reviews_bag_sample.pluck('rating').frequencies().compute()

8. Найдите среднее значение `rating` в выборке

In [ ]:
preproc_reviews_bag_sample.pluck('rating').mean().compute()

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [ ]:
preproc_reviews_bag_sample.foldby(
    lambda x: x['rating'],
    lambda prev_max, obj: max(prev_max, len(obj['review'])),
    initial=-1,
    combine=max,
    combine_initial=-1
).compute()